<a href="https://colab.research.google.com/github/medsellufbc/BMI/blob/main/SEIRDV_Dashboard_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.3 MB/s eta 0:00:00


In [3]:
!pip install dash plotly

In [7]:
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go

# SEIRDV model parameters
alpha = 0.2
gamma1 = 0.1
gamma2 = 0.1
gamma3 = 0.1
rho1 = 0.05
rho2 = 0.05
chi = 0.03
kappa = 0.01
lambda1 = 0.3
lambda2 = 0.2
lambda3 = 0.1
delta = 0.01  # Vaccination rate
tau = 0.1  # Vaccine efficacy

# Simulation settings
T = 356  # total time in days
dt = 0.1  # time step
n_steps = int(T / dt)

# Differential equations
def SEIRDV_model(x):
    S, E, I1, I2, I3, R, D, V = x

    dS = - (lambda1 * I1 + lambda2 * I2 + lambda3 * I3) * S
    dE = (lambda1 * I1 + lambda2 * I2 + lambda3 * I3) * S - alpha * E - kappa * E + tau * V * (lambda1 * I1 + lambda2 * I2 + lambda3 * I3)
    dI1 = alpha * E - gamma1 * I1 - rho1 * I1 - kappa * I1
    dI2 = rho1 * I1 - gamma2 * I2 - rho2 * I2 - kappa * I2
    dI3 = rho2 * I2 - gamma3 * I3 - chi * I3 - kappa * I3
    dR = gamma1 * I1 + gamma2 * I2 + gamma3 * I3 - kappa * R
    dD = chi * I3
    dV = S * delta - tau * V * (lambda1 * I1 + lambda2 * I2 + lambda3 * I3) - kappa * V

    return np.array([dS, dE, dI1, dI2, dI3, dR, dD, dV])

# Runge-Kutta 4th order method
def RK4_step(f, x, dt):
    k1 = dt * f(x)
    k2 = dt * f(x + 0.5 * k1)
    k3 = dt * f(x + 0.5 * k2)
    k4 = dt * f(x + k3)
    return x + (k1 + 2*k2 + 2*k3 + k4) / 6

# Create the Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("SEIRDV Model Simulation Dashboard"),
    html.Div([
        html.Div([
            html.Label("Initial Susceptible (S0)"),
            dcc.Input(id='S0-input', type='number', value=0.9, step=0.01),
            html.Label("Initial Exposed (E0)"),
            dcc.Input(id='E0-input', type='number', value=0.05, step=0.01),
            html.Label("Initial Infected 1 (I1_0)"),
            dcc.Input(id='I1_0-input', type='number', value=0.01, step=0.01),
            html.Label("Initial Infected 2 (I2_0)"),
            dcc.Input(id='I2_0-input', type='number', value=0.01, step=0.01),
            html.Label("Initial Infected 3 (I3_0)"),
            dcc.Input(id='I3_0-input', type='number', value=0.01, step=0.01),
            html.Label("Initial Recovered (R0)"),
            dcc.Input(id='R0-input', type='number', value=0.0, step=0.01),
            html.Label("Initial Deceased (D0)"),
            dcc.Input(id='D0-input', type='number', value=0.0, step=0.01),
            html.Label("Initial Vaccinated (V0)"),
            dcc.Input(id='V0-input', type='number', value=0.02, step=0.01),
            html.Button('Run Simulation', id='run-button', n_clicks=0)
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        html.Div([
            dcc.Graph(id='seirdv-graph')
        ], style={'width': '75%', 'display': 'inline-block'})
    ])
])

@app.callback(
    Output('seirdv-graph', 'figure'),
    [Input('run-button', 'n_clicks')],
    [State('S0-input', 'value'),
     State('E0-input', 'value'),
     State('I1_0-input', 'value'),
     State('I2_0-input', 'value'),
     State('I3_0-input', 'value'),
     State('R0-input', 'value'),
     State('D0-input', 'value'),
     State('V0-input', 'value')]
)
def update_graph(n_clicks, S0, E0, I1_0, I2_0, I3_0, R0, D0, V0):
    initial_conditions = np.array([S0, E0, I1_0, I2_0, I3_0, R0, D0, V0])

    # Initialize arrays to store results
    results = np.zeros((n_steps, 8))
    results[0] = initial_conditions

    # Simulation loop
    for t in range(1, n_steps):
        x = results[t-1]
        results[t] = RK4_step(SEIRDV_model, x, dt)

    time = np.linspace(0, T, n_steps)
    traces = []
    labels = ['S', 'E', 'I1', 'I2', 'I3', 'R', 'D', 'V']

    for i, label in enumerate(labels):
        traces.append(go.Scatter(
            x=time,
            y=results[:, i],
            mode='lines',
            name=label
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            title='SEIRDV Model Simulation (Runge-Kutta 4th Order Method)',
            xaxis={'title': 'Days'},
            yaxis={'title': 'Proportion of Population'},
            hovermode='closest'
        )
    }

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>